This is the initial notebook to explore the data and run the first models. 

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

from fastai.imports import *
from fastai.tabular.all import *

from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from scipy.stats import skew, norm
from catboost import CatBoostRegressor

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import shap
import xgboost
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from mlxtend.regressor import StackingCVRegressor

In [3]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

Based on initial analysis the train and test datasets have similar characteristics, so it will be easier to combine them for imputation and data analysis work. 

In [4]:
train_test = pd.concat([train, test], ignore_index=True)

In [5]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

It looks like Alley, FireplaceQu, PoolQC, Fence and MiscFeature have significant numbers of missing data. So it will be best to eliminate those rows. There are a number of rows that have less than 5 rows with missing data. Since some of these are categorical and some are continuous data, their missing data will be replaced with the most frequent value. 

Instead, for high_nan, replace the NaN with None; since they don't have that feature. 

In [6]:
drop_high_nan=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_test=train_test.drop(drop_high_nan,axis=1)

small_nan_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageCars', 
                  'GarageArea','SaleType', 'SaleCondition']
small_impute = SimpleImputer(strategy='most_frequent')
train_test[small_nan_cols] = pd.DataFrame(small_impute.fit_transform(train_test[small_nan_cols]),columns=small_nan_cols)

The following columns seem to have one value significantly larger than the rest, and it would probably be best to use the mode, or most common, value to feel each NaN value: MasVnrType, MasVnrArea, BsmtCond, BsmtExposure, BsmtFinType2, GarageType, GarageFinish, GarageQual, and GarageCond. That is represents 9 out of the 13 columns. 

BsmtQual has two values larger than the rest: Gd and TA. But it only has 2.8% NaNs, so simply using the mode might be good enough.  

GarageYrBlt has 59 NaNs out 2919 rows which is only 2%. It has a dispersed set of values, so it might be easiest just to have any NaNs have the same value as YearBuilt. 

BsmtFinType1 has only 2.7% value of NaNs, and most two of its largest values are GLQ and Unf. It might be easiest to use the mode here. 

LotFrontage has 486 NaNs out of 2919 rows which is a pretty high 16.7%. It has a dispersed range of values, but looking at its characteristics from the describe function above, it seems to have a pretty even distribution with a mean of 10,168 and a median of 9,453. So using the mean to fill in the NaNs seems reasonable. If it turns out this value has a high impact on the predictability of the SalePrice, then it might be good to revisit this assumption. 

In [7]:
mode_cols = ['MasVnrType', 'MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'GarageType', 'GarageFinish', 
             'GarageQual','GarageCond', 'BsmtQual', 'BsmtFinType1']
mode_impute = SimpleImputer(strategy='most_frequent')
train_test[mode_cols] = pd.DataFrame(mode_impute.fit_transform(train_test[mode_cols]),columns=mode_cols)
train_test['LotFrontage'].fillna((train_test['LotFrontage'].mean()), inplace=True)
train_test['GarageYrBlt'] = train_test['GarageYrBlt'].fillna(train_test['YearBuilt'])

In [8]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

SalePrice    1459
dtype: int64

In [9]:
train_test['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [10]:
train_test['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'LwQ'], dtype=object)

In [11]:
train_test.BsmtQual = train_test.BsmtQual.replace({"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0})

In [12]:
train_test.BsmtFinType1 = train_test.BsmtFinType1.replace({"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1,
                                                         "NA": 0})

To look at how many features have a skew above 0.5, since high skew can be an issue in regression analysis. 

In [13]:
number_cols = train_test.select_dtypes(include=np.number).columns.tolist()
number_cols

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtQual',
 'BsmtFinType1',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold',
 'SalePrice']

In [14]:
skew_features = train_test[number_cols].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skew_features[skew_features > 0.6]
skew_index = high_skew.index
skew_index

Index(['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', '3SsnPorch',
       'KitchenAbvGr', 'EnclosedPorch', 'ScreenPorch', 'OpenPorchSF',
       'WoodDeckSF', 'LotFrontage', '1stFlrSF', 'MSSubClass', 'GrLivArea',
       '2ndFlrSF', 'BsmtQual', 'TotRmsAbvGrd', 'Fireplaces', 'HalfBath'],
      dtype='object')

In [15]:
print("There are {} numerical features with Skew > 0.6 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skewness

There are 19 numerical features with Skew > 0.6 :


,Skew
MiscVal,21.947195
PoolArea,16.898328
LotArea,12.822431
LowQualFinSF,12.088761
3SsnPorch,11.376065
KitchenAbvGr,4.302254
EnclosedPorch,4.003891
ScreenPorch,3.946694
OpenPorchSF,2.535114
WoodDeckSF,1.842433


To normalize the features with skew above 0.5.

In [16]:
for i in skew_index:
    train_test[i] = np.log1p(train_test[i])

It will be necessary to identify all the columns that have non-numeric object values and then convert them to numeric values. 

In [17]:
obj_cols = list(train_test.select_dtypes(['object']).columns)

In [18]:
for column in obj_cols:
     train_test[column] = pd.factorize(train_test[column], sort=True)[0]

To create some new columns that might compound the effects of the existing columns with higher impact on the xgb score. 

In [19]:
train_test['QualCondSum'] = train_test['OverallQual'] + train_test['OverallCond']
train_test['RemodTime'] = train_test['YearRemodAdd'] - train_test['YearBuilt']
train_test['BsmtFinTypeSF1'] = train_test['BsmtFinType1'] * train_test['BsmtFinSF1']
train_test['TotalFlrSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF']
train_test['TotalFinSF'] = train_test['GrLivArea'] + train_test['BsmtFinSF1']
train_test['GarageCarArea'] = train_test['GarageArea'] * train_test['GarageCars']
train_test['TotalSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF'] + train_test['TotalBsmtSF']

To now eliminate the columns that have negative or zero influence on the xgb score. 

In [20]:
train_test.drop(['Utilities','Street','TotRmsAbvGrd','3SsnPorch','YrSold','Exterior2nd','LotConfig',
                              'HouseStyle','EnclosedPorch','WoodDeckSF','Foundation','RoofMatl','Electrical',
                              'GarageType','LotFrontage','SaleType','MoSold','BsmtExposure','1stFlrSF',
                              'BsmtFinSF1','Exterior1st','KitchenQual','TotalFlrSF'], axis=1, inplace=True)

To create a column with the log of the SalePrice to match the evaluation process in the contest. 

In [21]:
train_test['LogSalePrice'] = train_test['SalePrice'].apply(np.log)

WHY THESE FEATURES? HOW DOES IT WORK?

In [22]:
# Remove outliers
train_test.drop(train_test[(train_test['OverallQual']<5) & (train_test['LogSalePrice']>200000)].index, inplace=True)
train_test.drop(train_test[(train['GrLivArea']>4500) & (train_test['LogSalePrice']<300000)].index, inplace=True)
train_test.reset_index(drop=True, inplace=True)

To separate the train_test dataset back into the train and test datasets, identify the independent and dependent columns, and create the validation split.

In [23]:
train = train_test[train_test['SalePrice'].notnull()].copy()
test = train_test[train_test['SalePrice'].isnull()].drop(['SalePrice','LogSalePrice'],axis=1)
X = train.drop(['SalePrice','LogSalePrice'],axis=1)
y = train.LogSalePrice

In [24]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [25]:
# Setup cross validation folds
kf = KFold(n_splits=12, random_state=42, shuffle=True)

In [26]:
cat_model = CatBoostRegressor()

xgb_model = xgb.XGBRegressor(learning_rate=0.01,
                       n_estimators=6000,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:linear',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

lr_model = LinearRegression()

br_model = BayesianRidge(compute_score=True)

lightgbm_model = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=7000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)

svr_model = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003))

dtr_model = DecisionTreeRegressor()

rfr_model = RandomForestRegressor(n_estimators=1200,
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)

gbr_model = GradientBoostingRegressor(n_estimators=6000,
                                learning_rate=0.01,
                                max_depth=4,
                                max_features='sqrt',
                                min_samples_leaf=15,
                                min_samples_split=10,
                                loss='huber',
                                random_state=42)

ridge_alphas = [1e-15, 1e-10, 1e-8, 9e-4, 7e-4, 5e-4, 3e-4, 1e-4, 1e-3, 5e-2, 1e-2, 0.1, 0.3, 1, 3, 5, 10, 15, 
                18, 20, 30, 50, 75, 100]
ridge_model = make_pipeline(RobustScaler(), RidgeCV(alphas=ridge_alphas, cv=kf))

In [27]:
model_names = ['CatBoostRegressor','XGBRegressor','LinearRegression','BayesianRidge','LGBMRegressor',
               'SVR','DecisionTreeRegressor','RandomForestRegressor','GradientBoostingRegressor','RidgeRegressor']

In [28]:
models = [cat_model,xgb_model,lr_model,br_model,lightgbm_model,svr_model,dtr_model,rfr_model,gbr_model,ridge_model]

In [29]:
def rmse_cv(model,X,y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kf))
    return rmse

In [30]:
score_rmse = []
for i in models:
    rmse = np.sqrt(-cross_val_score(i, X, y, scoring="neg_mean_squared_error", cv=6))
    ave_rmse = np.mean(rmse)
    score_rmse.append(ave_rmse)

Learning rate set to 0.042222
0:	learn: 0.3859061	total: 148ms	remaining: 2m 28s
1:	learn: 0.3750498	total: 151ms	remaining: 1m 15s
2:	learn: 0.3646534	total: 153ms	remaining: 50.9s
3:	learn: 0.3549586	total: 155ms	remaining: 38.7s
4:	learn: 0.3456818	total: 158ms	remaining: 31.4s
5:	learn: 0.3374020	total: 161ms	remaining: 26.6s
6:	learn: 0.3294478	total: 163ms	remaining: 23.1s
7:	learn: 0.3212166	total: 165ms	remaining: 20.5s
8:	learn: 0.3135707	total: 168ms	remaining: 18.5s
9:	learn: 0.3060558	total: 170ms	remaining: 16.9s
10:	learn: 0.2991271	total: 172ms	remaining: 15.4s
11:	learn: 0.2927524	total: 174ms	remaining: 14.3s
12:	learn: 0.2862771	total: 177ms	remaining: 13.4s
13:	learn: 0.2797897	total: 179ms	remaining: 12.6s
14:	learn: 0.2732725	total: 182ms	remaining: 11.9s
15:	learn: 0.2678061	total: 184ms	remaining: 11.3s
16:	learn: 0.2623415	total: 186ms	remaining: 10.8s
17:	learn: 0.2571758	total: 189ms	remaining: 10.3s
18:	learn: 0.2515229	total: 192ms	remaining: 9.93s
19:	learn

180:	learn: 0.0952091	total: 584ms	remaining: 2.64s
181:	learn: 0.0950222	total: 587ms	remaining: 2.64s
182:	learn: 0.0948336	total: 589ms	remaining: 2.63s
183:	learn: 0.0946670	total: 592ms	remaining: 2.62s
184:	learn: 0.0944561	total: 594ms	remaining: 2.62s
185:	learn: 0.0943358	total: 597ms	remaining: 2.61s
186:	learn: 0.0941460	total: 599ms	remaining: 2.6s
187:	learn: 0.0940151	total: 602ms	remaining: 2.6s
188:	learn: 0.0938526	total: 605ms	remaining: 2.6s
189:	learn: 0.0937603	total: 608ms	remaining: 2.59s
190:	learn: 0.0936334	total: 611ms	remaining: 2.59s
191:	learn: 0.0934535	total: 613ms	remaining: 2.58s
192:	learn: 0.0932997	total: 616ms	remaining: 2.58s
193:	learn: 0.0931368	total: 620ms	remaining: 2.57s
194:	learn: 0.0929617	total: 623ms	remaining: 2.57s
195:	learn: 0.0927878	total: 626ms	remaining: 2.57s
196:	learn: 0.0925076	total: 629ms	remaining: 2.56s
197:	learn: 0.0923074	total: 632ms	remaining: 2.56s
198:	learn: 0.0921658	total: 634ms	remaining: 2.55s
199:	learn: 0.0

407:	learn: 0.0682723	total: 1.16s	remaining: 1.68s
408:	learn: 0.0681840	total: 1.16s	remaining: 1.68s
409:	learn: 0.0680650	total: 1.17s	remaining: 1.68s
410:	learn: 0.0679816	total: 1.17s	remaining: 1.67s
411:	learn: 0.0678257	total: 1.17s	remaining: 1.67s
412:	learn: 0.0677448	total: 1.17s	remaining: 1.67s
413:	learn: 0.0676072	total: 1.18s	remaining: 1.67s
414:	learn: 0.0674836	total: 1.18s	remaining: 1.66s
415:	learn: 0.0673975	total: 1.18s	remaining: 1.66s
416:	learn: 0.0673203	total: 1.19s	remaining: 1.66s
417:	learn: 0.0672126	total: 1.19s	remaining: 1.65s
418:	learn: 0.0671042	total: 1.19s	remaining: 1.65s
419:	learn: 0.0670027	total: 1.19s	remaining: 1.65s
420:	learn: 0.0669045	total: 1.19s	remaining: 1.64s
421:	learn: 0.0668165	total: 1.2s	remaining: 1.64s
422:	learn: 0.0667207	total: 1.2s	remaining: 1.64s
423:	learn: 0.0666005	total: 1.2s	remaining: 1.63s
424:	learn: 0.0664640	total: 1.2s	remaining: 1.63s
425:	learn: 0.0664566	total: 1.21s	remaining: 1.63s
426:	learn: 0.06

602:	learn: 0.0541650	total: 1.74s	remaining: 1.14s
603:	learn: 0.0540953	total: 1.74s	remaining: 1.14s
604:	learn: 0.0540354	total: 1.74s	remaining: 1.14s
605:	learn: 0.0539731	total: 1.74s	remaining: 1.13s
606:	learn: 0.0538463	total: 1.75s	remaining: 1.13s
607:	learn: 0.0537739	total: 1.75s	remaining: 1.13s
608:	learn: 0.0536831	total: 1.75s	remaining: 1.12s
609:	learn: 0.0536797	total: 1.75s	remaining: 1.12s
610:	learn: 0.0536421	total: 1.76s	remaining: 1.12s
611:	learn: 0.0535848	total: 1.76s	remaining: 1.12s
612:	learn: 0.0535807	total: 1.76s	remaining: 1.11s
613:	learn: 0.0535225	total: 1.77s	remaining: 1.11s
614:	learn: 0.0534745	total: 1.77s	remaining: 1.11s
615:	learn: 0.0533561	total: 1.77s	remaining: 1.11s
616:	learn: 0.0533276	total: 1.78s	remaining: 1.1s
617:	learn: 0.0532764	total: 1.78s	remaining: 1.1s
618:	learn: 0.0531698	total: 1.78s	remaining: 1.1s
619:	learn: 0.0530970	total: 1.78s	remaining: 1.09s
620:	learn: 0.0530626	total: 1.79s	remaining: 1.09s
621:	learn: 0.0

762:	learn: 0.0455917	total: 2.12s	remaining: 660ms
763:	learn: 0.0455446	total: 2.13s	remaining: 657ms
764:	learn: 0.0455410	total: 2.13s	remaining: 654ms
765:	learn: 0.0455057	total: 2.13s	remaining: 651ms
766:	learn: 0.0454603	total: 2.13s	remaining: 648ms
767:	learn: 0.0453701	total: 2.14s	remaining: 646ms
768:	learn: 0.0453257	total: 2.14s	remaining: 643ms
769:	learn: 0.0453079	total: 2.14s	remaining: 640ms
770:	learn: 0.0452397	total: 2.14s	remaining: 637ms
771:	learn: 0.0452087	total: 2.15s	remaining: 634ms
772:	learn: 0.0451833	total: 2.15s	remaining: 631ms
773:	learn: 0.0451801	total: 2.15s	remaining: 628ms
774:	learn: 0.0451761	total: 2.15s	remaining: 625ms
775:	learn: 0.0451670	total: 2.16s	remaining: 622ms
776:	learn: 0.0451191	total: 2.16s	remaining: 620ms
777:	learn: 0.0450788	total: 2.16s	remaining: 617ms
778:	learn: 0.0449907	total: 2.16s	remaining: 614ms
779:	learn: 0.0449420	total: 2.17s	remaining: 611ms
780:	learn: 0.0449094	total: 2.17s	remaining: 608ms
781:	learn: 

...

435:	learn: 0.0666975	total: 1.85s	remaining: 2.4s
436:	learn: 0.0666802	total: 1.86s	remaining: 2.4s
437:	learn: 0.0665923	total: 1.86s	remaining: 2.39s
438:	learn: 0.0665352	total: 1.87s	remaining: 2.39s
439:	learn: 0.0664532	total: 1.87s	remaining: 2.38s
440:	learn: 0.0664372	total: 1.88s	remaining: 2.38s
441:	learn: 0.0663747	total: 1.88s	remaining: 2.37s
442:	learn: 0.0662987	total: 1.88s	remaining: 2.37s
443:	learn: 0.0662284	total: 1.89s	remaining: 2.36s
444:	learn: 0.0661590	total: 1.89s	remaining: 2.36s
445:	learn: 0.0660418	total: 1.89s	remaining: 2.35s
446:	learn: 0.0659670	total: 1.9s	remaining: 2.35s
447:	learn: 0.0658689	total: 1.9s	remaining: 2.34s
448:	learn: 0.0658537	total: 1.91s	remaining: 2.34s
449:	learn: 0.0657723	total: 1.91s	remaining: 2.33s
450:	learn: 0.0656905	total: 1.91s	remaining: 2.33s
451:	learn: 0.0656170	total: 1.92s	remaining: 2.32s
452:	learn: 0.0654800	total: 1.92s	remaining: 2.32s
453:	learn: 0.0653680	total: 1.92s	remaining: 2.31s
454:	learn: 0.06

596:	learn: 0.0559631	total: 2.66s	remaining: 1.79s
597:	learn: 0.0558813	total: 2.66s	remaining: 1.79s
598:	learn: 0.0558081	total: 2.67s	remaining: 1.79s
599:	learn: 0.0557423	total: 2.68s	remaining: 1.79s
600:	learn: 0.0556682	total: 2.69s	remaining: 1.78s
601:	learn: 0.0556219	total: 2.69s	remaining: 1.78s
602:	learn: 0.0555737	total: 2.7s	remaining: 1.78s
603:	learn: 0.0555100	total: 2.7s	remaining: 1.77s
604:	learn: 0.0554393	total: 2.71s	remaining: 1.77s
605:	learn: 0.0553684	total: 2.72s	remaining: 1.77s
606:	learn: 0.0553144	total: 2.72s	remaining: 1.76s
607:	learn: 0.0552226	total: 2.73s	remaining: 1.76s
608:	learn: 0.0551220	total: 2.73s	remaining: 1.75s
609:	learn: 0.0550125	total: 2.74s	remaining: 1.75s
610:	learn: 0.0549427	total: 2.74s	remaining: 1.75s
611:	learn: 0.0549143	total: 2.75s	remaining: 1.74s
612:	learn: 0.0548661	total: 2.76s	remaining: 1.74s
613:	learn: 0.0548006	total: 2.76s	remaining: 1.74s
614:	learn: 0.0547259	total: 2.77s	remaining: 1.73s
615:	learn: 0.

768:	learn: 0.0464735	total: 3.67s	remaining: 1.1s
769:	learn: 0.0464237	total: 3.68s	remaining: 1.1s
770:	learn: 0.0463925	total: 3.68s	remaining: 1.09s
771:	learn: 0.0463380	total: 3.69s	remaining: 1.09s
772:	learn: 0.0463104	total: 3.7s	remaining: 1.09s
773:	learn: 0.0462838	total: 3.71s	remaining: 1.08s
774:	learn: 0.0462399	total: 3.71s	remaining: 1.08s
775:	learn: 0.0461819	total: 3.72s	remaining: 1.07s
776:	learn: 0.0461304	total: 3.72s	remaining: 1.07s
777:	learn: 0.0460639	total: 3.73s	remaining: 1.06s
778:	learn: 0.0460445	total: 3.73s	remaining: 1.06s
779:	learn: 0.0460135	total: 3.73s	remaining: 1.05s
780:	learn: 0.0459803	total: 3.74s	remaining: 1.05s
781:	learn: 0.0459067	total: 3.74s	remaining: 1.04s
782:	learn: 0.0458509	total: 3.75s	remaining: 1.04s
783:	learn: 0.0458218	total: 3.75s	remaining: 1.03s
784:	learn: 0.0457602	total: 3.76s	remaining: 1.03s
785:	learn: 0.0457097	total: 3.76s	remaining: 1.02s
786:	learn: 0.0456825	total: 3.77s	remaining: 1.02s
787:	learn: 0.0

931:	learn: 0.0395320	total: 4.48s	remaining: 327ms
932:	learn: 0.0395035	total: 4.49s	remaining: 323ms
933:	learn: 0.0394457	total: 4.5s	remaining: 318ms
934:	learn: 0.0393985	total: 4.51s	remaining: 313ms
935:	learn: 0.0393969	total: 4.51s	remaining: 309ms
936:	learn: 0.0393591	total: 4.52s	remaining: 304ms
937:	learn: 0.0393057	total: 4.52s	remaining: 299ms
938:	learn: 0.0392540	total: 4.53s	remaining: 294ms
939:	learn: 0.0392404	total: 4.54s	remaining: 289ms
940:	learn: 0.0391893	total: 4.54s	remaining: 285ms
941:	learn: 0.0391436	total: 4.54s	remaining: 280ms
942:	learn: 0.0391014	total: 4.55s	remaining: 275ms
943:	learn: 0.0390458	total: 4.56s	remaining: 271ms
944:	learn: 0.0390043	total: 4.57s	remaining: 266ms
945:	learn: 0.0389637	total: 4.58s	remaining: 261ms
946:	learn: 0.0389301	total: 4.58s	remaining: 256ms
947:	learn: 0.0388986	total: 4.58s	remaining: 252ms
948:	learn: 0.0388689	total: 4.59s	remaining: 247ms
949:	learn: 0.0388215	total: 4.6s	remaining: 242ms
950:	learn: 0.

Ill-conditioned matrix (rcond=3.2355e-20): result may not be accurate.
Ill-conditioned matrix (rcond=3.14127e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.99202e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.15831e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.03965e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.36022e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.05187e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.05837e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.04943e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.16945e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.26024e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.0601e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.92321e-17): result may not be accurate.
Ill-conditioned matrix (rcond=7.50743e-21): result may not be accu

In [31]:
model_scores = list(zip(model_names, score_rmse))

In [32]:
model_scores

[('CatBoostRegressor', 0.11411205350616899),
 ('XGBRegressor', 0.13748288476654888),
 ('LinearRegression', 0.11969691765411448),
 ('BayesianRidge', 0.12061212544953019),
 ('LGBMRegressor', 0.11915209696117295),
 ('SVR', 0.1246562847252383),
 ('DecisionTreeRegressor', 0.20477477860710278),
 ('RandomForestRegressor', 0.13853112867269568),
 ('GradientBoostingRegressor', 0.11615172234950873),
 ('RidgeRegressor', 0.11967179571435715)]

In [32]:
#model_scores

[('CatBoostRegressor', 0.11411205350616899),
 ('XGBRegressor', 0.13698185547470643),
 ('LinearRegression', 0.11969691765411448),
 ('BayesianRidge', 0.12061212544953019),
 ('LGBMRegressor', 0.1265171429674531),
 ('SVR', 0.2384458189370885),
 ('DecisionTreeRegressor', 0.20798226180188817),
 ('RandomForestRegressor', 0.13563910253987557),
 ('GradientBoostingRegressor', 0.12427935641474476)]

In [42]:
# Stack up all the models above except CatBoost, optimized using xgboost
stack_gen = StackingCVRegressor(regressors=(xgb_model,lr_model,br_model,lightgbm_model,svr_model,
                                            dtr_model,rfr_model,gbr_model,ridge_model),
                                meta_regressor=lr_model,
                                #meta_regressor=xgboost,
                                #random_state=42
                                use_features_in_secondary=True)

In [40]:
cat_model_all_data = cat_model.fit(X,y)
xgb_model_all_data = xgb_model.fit(X,y)
lr_model_all_data = lr_model.fit(X,y)
br_model_all_data = br_model.fit(X,y)
lightgbm_model_all_data = lightgbm_model.fit(X,y)
svr_model_all_data = svr_model.fit(X,y)
dtr_model_all_data = dtr_model.fit(X,y)
rfr_model_all_data = rfr_model.fit(X,y)
gbr_model_all_data = gbr_model.fit(X,y)
ridge_model_all_data = ridge_model.fit(X,y)

Learning rate set to 0.043456
0:	learn: 0.3878022	total: 8.88ms	remaining: 8.87s
1:	learn: 0.3766636	total: 17.4ms	remaining: 8.69s
2:	learn: 0.3661510	total: 23.7ms	remaining: 7.87s
3:	learn: 0.3564785	total: 30.7ms	remaining: 7.64s
4:	learn: 0.3465498	total: 36.6ms	remaining: 7.29s
5:	learn: 0.3370452	total: 42.3ms	remaining: 7s
6:	learn: 0.3282236	total: 47.1ms	remaining: 6.69s
7:	learn: 0.3202490	total: 52.4ms	remaining: 6.5s
8:	learn: 0.3122182	total: 57.2ms	remaining: 6.3s
9:	learn: 0.3044626	total: 62.8ms	remaining: 6.21s
10:	learn: 0.2974128	total: 67.3ms	remaining: 6.05s
11:	learn: 0.2900735	total: 71.1ms	remaining: 5.85s
12:	learn: 0.2835559	total: 75.1ms	remaining: 5.71s
13:	learn: 0.2769941	total: 79.3ms	remaining: 5.58s
14:	learn: 0.2705944	total: 84.1ms	remaining: 5.53s
15:	learn: 0.2640226	total: 88.6ms	remaining: 5.45s
16:	learn: 0.2581572	total: 94.1ms	remaining: 5.44s
17:	learn: 0.2526829	total: 99.6ms	remaining: 5.43s
18:	learn: 0.2468738	total: 104ms	remaining: 5.36

160:	learn: 0.0972326	total: 808ms	remaining: 4.21s
161:	learn: 0.0971368	total: 813ms	remaining: 4.2s
162:	learn: 0.0969333	total: 818ms	remaining: 4.2s
163:	learn: 0.0967122	total: 823ms	remaining: 4.19s
164:	learn: 0.0965112	total: 828ms	remaining: 4.19s
165:	learn: 0.0963812	total: 835ms	remaining: 4.2s
166:	learn: 0.0962244	total: 840ms	remaining: 4.19s
167:	learn: 0.0960185	total: 845ms	remaining: 4.18s
168:	learn: 0.0958248	total: 852ms	remaining: 4.19s
169:	learn: 0.0956538	total: 858ms	remaining: 4.19s
170:	learn: 0.0955217	total: 862ms	remaining: 4.18s
171:	learn: 0.0954098	total: 867ms	remaining: 4.17s
172:	learn: 0.0952872	total: 872ms	remaining: 4.17s
173:	learn: 0.0950582	total: 877ms	remaining: 4.16s
174:	learn: 0.0948942	total: 882ms	remaining: 4.16s
175:	learn: 0.0947865	total: 887ms	remaining: 4.15s
176:	learn: 0.0946510	total: 891ms	remaining: 4.14s
177:	learn: 0.0944983	total: 896ms	remaining: 4.14s
178:	learn: 0.0943378	total: 902ms	remaining: 4.14s
179:	learn: 0.0

334:	learn: 0.0758277	total: 1.84s	remaining: 3.66s
335:	learn: 0.0757331	total: 1.85s	remaining: 3.66s
336:	learn: 0.0757102	total: 1.86s	remaining: 3.66s
337:	learn: 0.0755865	total: 1.87s	remaining: 3.67s
338:	learn: 0.0755744	total: 1.89s	remaining: 3.68s
339:	learn: 0.0755105	total: 1.9s	remaining: 3.68s
340:	learn: 0.0754122	total: 1.9s	remaining: 3.68s
341:	learn: 0.0753668	total: 1.91s	remaining: 3.68s
342:	learn: 0.0752662	total: 1.92s	remaining: 3.67s
343:	learn: 0.0751426	total: 1.92s	remaining: 3.67s
344:	learn: 0.0750395	total: 1.93s	remaining: 3.67s
345:	learn: 0.0749470	total: 1.94s	remaining: 3.66s
346:	learn: 0.0747750	total: 1.95s	remaining: 3.67s
347:	learn: 0.0747379	total: 1.95s	remaining: 3.66s
348:	learn: 0.0746428	total: 1.96s	remaining: 3.66s
349:	learn: 0.0745612	total: 1.97s	remaining: 3.65s
350:	learn: 0.0744530	total: 1.97s	remaining: 3.65s
351:	learn: 0.0743769	total: 1.98s	remaining: 3.65s
352:	learn: 0.0742679	total: 1.99s	remaining: 3.65s
353:	learn: 0.

511:	learn: 0.0619362	total: 2.88s	remaining: 2.74s
512:	learn: 0.0618730	total: 2.88s	remaining: 2.74s
513:	learn: 0.0618674	total: 2.89s	remaining: 2.73s
514:	learn: 0.0617386	total: 2.9s	remaining: 2.73s
515:	learn: 0.0616368	total: 2.9s	remaining: 2.73s
516:	learn: 0.0615789	total: 2.91s	remaining: 2.72s
517:	learn: 0.0615006	total: 2.92s	remaining: 2.72s
518:	learn: 0.0614463	total: 2.92s	remaining: 2.71s
519:	learn: 0.0613629	total: 2.93s	remaining: 2.71s
520:	learn: 0.0613038	total: 2.94s	remaining: 2.7s
521:	learn: 0.0612243	total: 2.94s	remaining: 2.69s
522:	learn: 0.0611420	total: 2.95s	remaining: 2.69s
523:	learn: 0.0610730	total: 2.95s	remaining: 2.68s
524:	learn: 0.0610264	total: 2.96s	remaining: 2.67s
525:	learn: 0.0609518	total: 2.96s	remaining: 2.67s
526:	learn: 0.0608858	total: 2.97s	remaining: 2.66s
527:	learn: 0.0608752	total: 2.97s	remaining: 2.65s
528:	learn: 0.0608236	total: 2.98s	remaining: 2.65s
529:	learn: 0.0607651	total: 2.98s	remaining: 2.64s
530:	learn: 0.0

701:	learn: 0.0519292	total: 3.89s	remaining: 1.65s
702:	learn: 0.0518798	total: 3.9s	remaining: 1.65s
703:	learn: 0.0518559	total: 3.9s	remaining: 1.64s
704:	learn: 0.0518531	total: 3.91s	remaining: 1.64s
705:	learn: 0.0517897	total: 3.92s	remaining: 1.63s
706:	learn: 0.0517284	total: 3.94s	remaining: 1.63s
707:	learn: 0.0517005	total: 3.95s	remaining: 1.63s
708:	learn: 0.0516543	total: 3.96s	remaining: 1.62s
709:	learn: 0.0515714	total: 3.98s	remaining: 1.63s
710:	learn: 0.0515582	total: 4s	remaining: 1.62s
711:	learn: 0.0515197	total: 4.01s	remaining: 1.62s
712:	learn: 0.0514626	total: 4.02s	remaining: 1.62s
713:	learn: 0.0513886	total: 4.03s	remaining: 1.61s
714:	learn: 0.0513338	total: 4.04s	remaining: 1.61s
715:	learn: 0.0512714	total: 4.06s	remaining: 1.61s
716:	learn: 0.0511952	total: 4.07s	remaining: 1.61s
717:	learn: 0.0511418	total: 4.08s	remaining: 1.6s
718:	learn: 0.0510966	total: 4.09s	remaining: 1.6s
719:	learn: 0.0510214	total: 4.11s	remaining: 1.6s
720:	learn: 0.050983

899:	learn: 0.0435103	total: 5.11s	remaining: 568ms
900:	learn: 0.0434692	total: 5.11s	remaining: 562ms
901:	learn: 0.0434590	total: 5.12s	remaining: 556ms
902:	learn: 0.0434087	total: 5.12s	remaining: 551ms
903:	learn: 0.0433635	total: 5.13s	remaining: 545ms
904:	learn: 0.0433142	total: 5.14s	remaining: 539ms
905:	learn: 0.0432754	total: 5.14s	remaining: 534ms
906:	learn: 0.0432293	total: 5.15s	remaining: 528ms
907:	learn: 0.0431562	total: 5.16s	remaining: 523ms
908:	learn: 0.0431449	total: 5.16s	remaining: 517ms
909:	learn: 0.0431066	total: 5.17s	remaining: 511ms
910:	learn: 0.0430719	total: 5.17s	remaining: 505ms
911:	learn: 0.0430700	total: 5.18s	remaining: 500ms
912:	learn: 0.0430170	total: 5.18s	remaining: 494ms
913:	learn: 0.0429739	total: 5.19s	remaining: 488ms
914:	learn: 0.0429369	total: 5.2s	remaining: 483ms
915:	learn: 0.0429343	total: 5.2s	remaining: 477ms
916:	learn: 0.0428880	total: 5.21s	remaining: 472ms
917:	learn: 0.0428863	total: 5.22s	remaining: 466ms
918:	learn: 0.

Ill-conditioned matrix (rcond=9.87008e-20): result may not be accurate.
Ill-conditioned matrix (rcond=4.0889e-20): result may not be accurate.
Ill-conditioned matrix (rcond=5.77396e-20): result may not be accurate.
Ill-conditioned matrix (rcond=2.61217e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.56899e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.50738e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.61955e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.63965e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.58269e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.71824e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.77221e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.5599e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.65884e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.5439e-17): result may not be accur

In [50]:
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))

[17:14:57] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[17:15:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[17:15:11] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[17:15:18] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[17:15:26] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscal

Ill-conditioned matrix (rcond=6.19981e-20): result may not be accurate.
Ill-conditioned matrix (rcond=3.53737e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.31964e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.23875e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.40109e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.31699e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.39995e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.54485e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.38754e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.36539e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.42689e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.29782e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.34509e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.20615e-20): result may not be ac

[17:18:07] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


Ill-conditioned matrix (rcond=9.87008e-20): result may not be accurate.
Ill-conditioned matrix (rcond=4.0889e-20): result may not be accurate.
Ill-conditioned matrix (rcond=5.77396e-20): result may not be accurate.
Ill-conditioned matrix (rcond=2.61217e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.56899e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.50738e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.61955e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.63965e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.58269e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.71824e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.77221e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.5599e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.65884e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.5439e-17): result may not be accur

In [43]:
print(rmse_cv(stack_gen,np.array(X),np.array(y)).mean())

[15:38:28] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:38:40] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:38:51] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:39:03] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:39:14] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscal

Ill-conditioned matrix (rcond=4.62111e-20): result may not be accurate.
Ill-conditioned matrix (rcond=1.30784e-19): result may not be accurate.
Ill-conditioned matrix (rcond=3.37273e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.54987e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.49751e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.61938e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.66082e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.60819e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.61142e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.51381e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.87689e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.92519e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.86004e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.42896e-17): result may not be ac

[15:44:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


Ill-conditioned matrix (rcond=4.34027e-20): result may not be accurate.
Ill-conditioned matrix (rcond=3.08508e-21): result may not be accurate.
Ill-conditioned matrix (rcond=2.9044e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.84148e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.8943e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.80761e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.78832e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.98754e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.81084e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.84531e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.97747e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.83958e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.7936e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.87048e-17): result may not be accur

[15:45:36] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:46:12] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:46:48] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:47:28] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[15:48:04] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscal

...

[16:52:34] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[16:52:44] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[16:52:53] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[16:53:02] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[16:53:11] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscal

Ill-conditioned matrix (rcond=5.04297e-20): result may not be accurate.
Ill-conditioned matrix (rcond=7.33967e-20): result may not be accurate.
Ill-conditioned matrix (rcond=3.75336e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.58045e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.63216e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.51794e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.67525e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.52337e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.84071e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.505e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.95133e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.63321e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.56109e-17): result may not be accurate.
Ill-conditioned matrix (rcond=3.74293e-17): result may not be accu

[16:55:49] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-0fc7796c793e6356f-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[LightGBM] [Warning] feature_fraction is set=0.2, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=4, subsample_freq=0 will be ignored. Current value: bagging_freq=4


Ill-conditioned matrix (rcond=4.21879e-20): result may not be accurate.
Ill-conditioned matrix (rcond=4.22061e-20): result may not be accurate.
Ill-conditioned matrix (rcond=4.31881e-20): result may not be accurate.
Ill-conditioned matrix (rcond=2.67436e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.75267e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.71265e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.6218e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.8346e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.8766e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.73436e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.59524e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.64481e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.71288e-17): result may not be accurate.
Ill-conditioned matrix (rcond=2.95214e-17): result may not be accur

0.1185237874666849


In [47]:
# Blend models in order to make the final predictions more robust to overfitting
def blended_predictions(X):
    return ((0.1 * ridge_model_all_data.predict(X)) + \
            (0.2 * svr_model_all_data.predict(X)) + \
            (0.1 * gbr_model_all_data.predict(X)) + \
            (0.1 * xgb_model_all_data.predict(X)) + \
            (0.1 * lightgbm_model_all_data.predict(X)) + \
            (0.05 * rfr_model_all_data.predict(X)) + \
            (0.35 * stack_gen_model.predict(np.array(X))))

In [48]:
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [52]:
# Get final precitions from the blended model
blended_score = rmsle(y, blended_predictions(X))
#scores['blended'] = (blended_score, 0)
print('RMLSE score on train data:')
print(blended_score)

RMLSE score on train data:
0.08601190532463165


In [33]:
submit = test[['Id']]
submit = submit.reset_index(drop=True)

In [34]:
cat_model.fit(X,y)

Learning rate set to 0.043456
0:	learn: 0.3878022	total: 4.7ms	remaining: 4.7s
1:	learn: 0.3766636	total: 8.34ms	remaining: 4.16s
2:	learn: 0.3661510	total: 11.6ms	remaining: 3.87s
3:	learn: 0.3564785	total: 15.2ms	remaining: 3.79s
4:	learn: 0.3465498	total: 18.9ms	remaining: 3.76s
5:	learn: 0.3370452	total: 22.2ms	remaining: 3.68s
6:	learn: 0.3282236	total: 26.2ms	remaining: 3.72s
7:	learn: 0.3202490	total: 29.4ms	remaining: 3.64s
8:	learn: 0.3122182	total: 31.9ms	remaining: 3.51s
9:	learn: 0.3044626	total: 34.4ms	remaining: 3.4s
10:	learn: 0.2974128	total: 36.8ms	remaining: 3.3s
11:	learn: 0.2900735	total: 39.6ms	remaining: 3.26s
12:	learn: 0.2835559	total: 42.7ms	remaining: 3.24s
13:	learn: 0.2769941	total: 46.6ms	remaining: 3.28s
14:	learn: 0.2705944	total: 49.5ms	remaining: 3.25s
15:	learn: 0.2640226	total: 52.4ms	remaining: 3.22s
16:	learn: 0.2581572	total: 55.3ms	remaining: 3.2s
17:	learn: 0.2526829	total: 58.5ms	remaining: 3.19s
18:	learn: 0.2468738	total: 61.4ms	remaining: 3.1

207:	learn: 0.0902901	total: 581ms	remaining: 2.21s
208:	learn: 0.0901098	total: 586ms	remaining: 2.22s
209:	learn: 0.0899392	total: 589ms	remaining: 2.21s
210:	learn: 0.0897029	total: 592ms	remaining: 2.21s
211:	learn: 0.0895092	total: 595ms	remaining: 2.21s
212:	learn: 0.0893500	total: 598ms	remaining: 2.21s
213:	learn: 0.0892382	total: 602ms	remaining: 2.21s
214:	learn: 0.0890623	total: 606ms	remaining: 2.21s
215:	learn: 0.0890047	total: 610ms	remaining: 2.21s
216:	learn: 0.0889241	total: 613ms	remaining: 2.21s
217:	learn: 0.0887979	total: 615ms	remaining: 2.21s
218:	learn: 0.0887074	total: 619ms	remaining: 2.21s
219:	learn: 0.0886302	total: 622ms	remaining: 2.21s
220:	learn: 0.0885188	total: 625ms	remaining: 2.2s
221:	learn: 0.0883617	total: 628ms	remaining: 2.2s
222:	learn: 0.0882470	total: 631ms	remaining: 2.2s
223:	learn: 0.0881239	total: 634ms	remaining: 2.2s
224:	learn: 0.0880452	total: 637ms	remaining: 2.19s
225:	learn: 0.0878952	total: 641ms	remaining: 2.19s
226:	learn: 0.08

386:	learn: 0.0710582	total: 1.16s	remaining: 1.84s
387:	learn: 0.0709672	total: 1.16s	remaining: 1.84s
388:	learn: 0.0708850	total: 1.17s	remaining: 1.83s
389:	learn: 0.0708290	total: 1.17s	remaining: 1.83s
390:	learn: 0.0707294	total: 1.17s	remaining: 1.83s
391:	learn: 0.0706576	total: 1.18s	remaining: 1.83s
392:	learn: 0.0705101	total: 1.18s	remaining: 1.82s
393:	learn: 0.0704446	total: 1.18s	remaining: 1.82s
394:	learn: 0.0703348	total: 1.19s	remaining: 1.82s
395:	learn: 0.0703151	total: 1.19s	remaining: 1.82s
396:	learn: 0.0702307	total: 1.19s	remaining: 1.81s
397:	learn: 0.0701661	total: 1.2s	remaining: 1.81s
398:	learn: 0.0701237	total: 1.2s	remaining: 1.81s
399:	learn: 0.0700386	total: 1.2s	remaining: 1.81s
400:	learn: 0.0699385	total: 1.21s	remaining: 1.8s
401:	learn: 0.0698492	total: 1.21s	remaining: 1.8s
402:	learn: 0.0697730	total: 1.21s	remaining: 1.8s
403:	learn: 0.0697439	total: 1.22s	remaining: 1.79s
404:	learn: 0.0696497	total: 1.22s	remaining: 1.79s
405:	learn: 0.0695

584:	learn: 0.0575651	total: 1.75s	remaining: 1.24s
585:	learn: 0.0574954	total: 1.76s	remaining: 1.24s
586:	learn: 0.0574228	total: 1.76s	remaining: 1.24s
587:	learn: 0.0573762	total: 1.76s	remaining: 1.24s
588:	learn: 0.0573121	total: 1.76s	remaining: 1.23s
589:	learn: 0.0572500	total: 1.77s	remaining: 1.23s
590:	learn: 0.0572048	total: 1.77s	remaining: 1.23s
591:	learn: 0.0570968	total: 1.77s	remaining: 1.22s
592:	learn: 0.0570902	total: 1.78s	remaining: 1.22s
593:	learn: 0.0570484	total: 1.78s	remaining: 1.22s
594:	learn: 0.0570400	total: 1.78s	remaining: 1.21s
595:	learn: 0.0569855	total: 1.79s	remaining: 1.21s
596:	learn: 0.0569124	total: 1.79s	remaining: 1.21s
597:	learn: 0.0568743	total: 1.79s	remaining: 1.21s
598:	learn: 0.0568265	total: 1.79s	remaining: 1.2s
599:	learn: 0.0567767	total: 1.8s	remaining: 1.2s
600:	learn: 0.0567719	total: 1.8s	remaining: 1.2s
601:	learn: 0.0567615	total: 1.8s	remaining: 1.19s
602:	learn: 0.0566707	total: 1.81s	remaining: 1.19s
603:	learn: 0.0565

795:	learn: 0.0476771	total: 2.34s	remaining: 599ms
796:	learn: 0.0476492	total: 2.34s	remaining: 596ms
797:	learn: 0.0475860	total: 2.34s	remaining: 594ms
798:	learn: 0.0475820	total: 2.35s	remaining: 591ms
799:	learn: 0.0475403	total: 2.35s	remaining: 588ms
800:	learn: 0.0474602	total: 2.35s	remaining: 585ms
801:	learn: 0.0474157	total: 2.36s	remaining: 582ms
802:	learn: 0.0473189	total: 2.36s	remaining: 579ms
803:	learn: 0.0472559	total: 2.36s	remaining: 576ms
804:	learn: 0.0472524	total: 2.37s	remaining: 573ms
805:	learn: 0.0472197	total: 2.37s	remaining: 571ms
806:	learn: 0.0471809	total: 2.37s	remaining: 568ms
807:	learn: 0.0471465	total: 2.38s	remaining: 565ms
808:	learn: 0.0470991	total: 2.38s	remaining: 562ms
809:	learn: 0.0470957	total: 2.38s	remaining: 559ms
810:	learn: 0.0470688	total: 2.39s	remaining: 556ms
811:	learn: 0.0470173	total: 2.39s	remaining: 553ms
812:	learn: 0.0470139	total: 2.39s	remaining: 550ms
813:	learn: 0.0469692	total: 2.39s	remaining: 547ms
814:	learn: 

993:	learn: 0.0400002	total: 2.92s	remaining: 17.6ms
994:	learn: 0.0399989	total: 2.93s	remaining: 14.7ms
995:	learn: 0.0399743	total: 2.94s	remaining: 11.8ms
996:	learn: 0.0399729	total: 2.95s	remaining: 8.87ms
997:	learn: 0.0399371	total: 2.95s	remaining: 5.92ms
998:	learn: 0.0398792	total: 2.96s	remaining: 2.96ms
999:	learn: 0.0398716	total: 2.96s	remaining: 0us


In [35]:
submit_predict = cat_model.predict(test)
submit_predict = np.exp(submit_predict)

In [36]:
submit['SalePrice'] = submit_predict

In [37]:
submit.to_csv('submit_cat-03-models.csv', index=False)

CatBoost with default parameters model=0.11411 Kaggle 0.12926
